In [1]:
import os
os.chdir("../")

In [2]:

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sajithseelan17/Named-Entity-Recognition-News.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sajithseelan17"
os.environ["MLFLOW_TRACKING_PASSWORD"]="194da69e16c5534858029590ccff6c5e6a34d178"

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    mlflow_uri: str
    all_params: dict
    params_num_encoder_layers: int
    params_num_tags: int
    params_vocab_size: int
    params_embedding_dim: int
    params_fully_connected_dim: int
    params_num_heads: int
    params_max_positional_encoding_length: int
    params_batch_size: int

In [4]:
from named_entity_recognition.constants import *
from named_entity_recognition.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self):

        config = self.config.model_evaluation
        create_directories([config.root_dir])

        evaluation_config = ModelEvaluationConfig(
            root_dir = Path(config.root_dir),
            data_path = Path(config.data_path),
            model_path = Path(config.model_path),
            mlflow_uri = config.mlflow_uri,
            all_params = self.params,
            params_num_encoder_layers=self.params.NUM_ENCODER_LAYERS,
            params_num_tags = self.params.NUM_TAGS,
            params_vocab_size = self.params.VOCAB_SIZE,
            params_embedding_dim = self.params.EMBEDDING_DIM,
            params_fully_connected_dim = self.params.FULLY_CONNECTED_DIM,
            params_num_heads = self.params.NUM_HEADS,
            params_max_positional_encoding_length = self.params.MAX_POSITIONAL_ENCODING_LENGTH,
            params_batch_size = self.params.BATCH_SIZE,    
        )

        return evaluation_config

In [6]:
import os
import json
from datasets import load_from_disk
from named_entity_recognition import logger
from named_entity_recognition.transformer.model import NERModel
from named_entity_recognition.transformer.losses import MaskedLoss
from named_entity_recognition.transformer.metrics import masked_acc
from named_entity_recognition.transformer.optimizer import CustomSchedule
from named_entity_recognition.utils.common import save_json
import numpy as np
import tensorflow as tf
import mlflow
from urllib.parse import urlparse
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

[2024-04-27 16:06:11,203: INFO: config: TensorFlow version 2.14.0 available.]


C:\Users\sajit\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
class ModelEvaluation:

    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate(self):
        
        test = load_from_disk(str(self.config.data_path))['test']
        test_input = tf.data.Dataset.from_tensor_slices(tf.concat((test['input_ids'],test['attention_mask']), axis = -1)).batch(self.config.params_batch_size)
        y_true = test['labels']
        y_pred = self.model.predict(test_input)
        loss = MaskedLoss()(y_true, y_pred)
        metrics = self.compute_metrics(y_true, y_pred)

        self.scores = {
            'loss': float(loss),
            **metrics,
        }
        save_json(Path(os.path.join(self.config.root_dir,'scores.json')), self.scores)

    def get_model(self):
        self.model = tf.keras.models.load_model(self.config.model_path, custom_objects={
            'NERModel':NERModel,
            'MaskedLoss':MaskedLoss,
            'masked_acc': masked_acc,
            'CustomSchedule': CustomSchedule
        })

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                self.scores
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model.keras", registered_model_name="NERModel")
            else:
                mlflow.keras.log_model(self.model, "model.keras")
        

    @staticmethod
    def compute_metrics(labels, pred_logits):

        pred_logits = np.argmax(pred_logits, axis=-1)

        predictions = []
        for prediction, label in zip(pred_logits, labels):
            for eval_pred, l in zip(prediction, label):
                if l!= -1:
                    predictions.append(eval_pred)

        true_labels = []
        for prediction, label in zip(pred_logits, labels):
            for eval_pred, l in zip(prediction, label):
                if l!= -1:
                    true_labels.append(l)

        accuracy = accuracy_score(true_labels, predictions)
        precision = precision_score(true_labels, predictions, average='macro')
        recall = recall_score(true_labels, predictions, average='macro')
        f1 = f1_score(true_labels, predictions, average='macro')
        
        return {'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1_score': f1 }


In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.get_model()
    model_evaluation.evaluate()
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-04-27 16:09:27,518: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-27 16:09:27,522: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-27 16:09:27,523: INFO: common: created directory at: artifacts]
[2024-04-27 16:09:27,524: INFO: common: created directory at: artifacts/model_evaluation]
54/54 [==============================] - 4s 61ms/step
[2024-04-27 16:09:32,867: INFO: common: json file saved at: artifacts\model_evaluation\scores.json]


c:\Users\sajit\anaconda3\envs\NER\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2024/04/27 16:09:34 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-04-27 16:09:42,409: INFO: builder_impl: Assets written to: C:\Users\sajit\AppData\Local\Temp\tmp6dc1a1lo\model\data\model\assets]


2024/04/27 16:09:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\sajit\AppData\Local\Temp\tmp6dc1a1lo\model, flavor: tensorflow). Fall back to return ['tensorflow==2.14.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
c:\Users\sajit\anaconda3\envs\NER\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'Model'.
2024/04/27 16:10:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Model, version 1
Created version '1' of model 'Model'.
